In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import EarlyStopping
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
hotel_nyc_raw=pd.read_csv('/content/gdrive/MyDrive/AB_NYC_2019.csv',delimiter=',')

In [13]:
hotel_nyc = hotel_nyc_raw[hotel_nyc_raw.loc[:]!=0].dropna()

In [14]:
hotel_nyc=hotel_nyc.dropna()

In [15]:
hotel_nyc.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149.0,1,9.0,2018-10-19,0.21,6,365.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225.0,1,45.0,2019-05-21,0.38,2,355.0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89.0,1,270.0,2019-07-05,4.64,1,194.0
5,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200.0,3,74.0,2019-06-22,0.59,1,129.0
7,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79.0,2,430.0,2019-06-24,3.47,1,220.0


In [16]:
hotel_nyc.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [17]:
def neighbourhood_groupbased_review(neighbourhood_group):
    hotel_nyc['neighbourhood_group']=hotel_nyc['neighbourhood_group'].str.lower()
    neighbourhood_groupbase=hotel_nyc[hotel_nyc['neighbourhood_group']==neighbourhood_group.lower()]
    neighbourhood_groupbase=neighbourhood_groupbase.sort_values(by='reviews_per_month',ascending=False)
    neighbourhood_groupbase.drop_duplicates(subset='host_id',keep='first',inplace=True)
    if(neighbourhood_groupbase.empty==0):
        hname=neighbourhood_groupbase[['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365']]
        return hname.head()
    else:
        print('No Hotels Available')

In [18]:
print('Recommended hotels by reviews per month')
neighbourhood_groupbased_review('Manhattan')

Recommended hotels by reviews per month


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
42075,32678719,Enjoy great views of the City in our Deluxe Room!,244361589,Row NYC,manhattan,Theater District,40.75918,-73.98801,Private room,100.0,1,156.0,2019-07-07,58.50,9,299.0
46937,35495582,★ AMAZING★ TIME SQUARE/ 2 Bedroom 3 Bed Apart...,266992480,Sam,manhattan,Hell's Kitchen,40.76573,-73.98897,Entire home/apt,500.0,3,14.0,2019-07-07,14.00,2,45.0
41954,32611321,Amazing apartment close to midtown with skylight,27287203,Ben,manhattan,Upper West Side,40.79887,-73.96343,Private room,80.0,1,42.0,2019-07-07,13.13,1,16.0
33943,26877259,*Times Square Luxury Apartment Private Room*,172864847,Jae,manhattan,Theater District,40.75964,-73.98731,Private room,100.0,1,137.0,2019-07-07,12.05,1,36.0
46891,35480817,★★★Chic place in center of Manhattan★★★,266828681,Alex,manhattan,Hell's Kitchen,40.75493,-73.99458,Entire home/apt,320.0,1,12.0,2019-07-02,12.00,1,189.0


In [19]:
def neighbourhood_groupbased_price(neighbourhood_group):
    hotel_nyc['neighbourhood_group']=hotel_nyc['neighbourhood_group'].str.lower()
    neighbourhood_groupbase=hotel_nyc[hotel_nyc['neighbourhood_group']==neighbourhood_group.lower()]
    neighbourhood_groupbase=neighbourhood_groupbase.sort_values(by='price',ascending=True)
    neighbourhood_groupbase.drop_duplicates(subset='host_id',keep='first',inplace=True)
    neighbourhood_groupbase.drop_duplicates(subset='host_id',keep='first',inplace=True)
    if(neighbourhood_groupbase.empty==0):
        hname=neighbourhood_groupbase[['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365']]
        return hname.head()
    else:
        print('No Hotels Available')

In [20]:
print('Recommended hotels by cheapest price')
neighbourhood_groupbased_price('Brooklyn')

Recommended hotels by cheapest price


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
32810,25839759,Gigantic Sunny Room in Park Slope-Private Back...,167570251,Rachel,brooklyn,Sunset Park,40.66242,-73.99464,Entire home/apt,10.0,1,14.0,2018-10-28,1.06,1,4.0
27972,21869057,Spacious 2-bedroom Apt in Heart of Greenpoint,11967922,Vishanti & Jeremy,brooklyn,Greenpoint,40.72421,-73.95364,Entire home/apt,10.0,1,93.0,2019-07-01,4.73,1,32.0
20992,16620607,Spacious and Modern 2 Bedroom Apartment,109725962,Erika,brooklyn,Bushwick,40.68994,-73.91556,Entire home/apt,11.0,2,113.0,2019-06-22,3.86,1,261.0
28841,22258432,Awesome designed shared room in beautiful Coli...,146345538,Sergii,brooklyn,Bushwick,40.69329,-73.91328,Shared room,15.0,30,3.0,2018-07-14,0.16,5,344.0
39027,30489379,Calm bed. perfect for students and travelers.,111542307,Imoy,brooklyn,Bedford-Stuyvesant,40.68622,-73.92639,Shared room,19.0,30,1.0,2019-01-09,0.17,2,339.0
